# Описание задания

В рамках данного задания, студент должен создать и оценить 4 типа рекомендательных систем:
* Item-based collaborative filtering RS


# Детальное описание

#### Данные: 
Датасет представлен множеством отзывов к компьютерным играм (объектам) от пользователей Amazon. Каждый отзыв представлен в виде JSON-структуры со следующими полями:
* идентификатор пользователя - reviewerID
* идентификатор объекта - asin
* текст отзыва - reviewText
* рейтинг - overall
* время публикации обзора - unixReviewTime
* другие поля, не использованные автором этого блокнота (смотри полное описание JSON [тут](http://jmcauley.ucsd.edu/data/amazon/))

У каждого объекта есть как минимум 5 отзывов, каждый пользователь написал как минимум 5 отзывов. 
#### Цель: 
Построить рекомендательную систему, предсказывающую объекты, которые пользователь приобретет в ближайшем будущем. Для упрощения мы считаем, что пользователь приобрел объект, если он написал про него отзыв.
#### Подготовка данных:
Данные разделены на тренировочную и тестовую выборки по времени публикации отзывов. Первые 80% данных (более старые) используются как тренировочная выборка, остальные - как тестовая. 

Построение рекомендательной системы (т.е., выбор и тренировка моделей, оптимизация параметров и т.д.) осуществляется **только** с использованием тренировочной выборки. Все параметры, использованные в моделях, **должны быть** получены или объяснены с помощью тренировочных данных. Студент вправе использовать тренировочную выборку как его душе угодно. 

Тестирующая выборка используется **только** для оценки рекомендательной системы.

Для построения рекомендательных моделей также можно использовать JSON-поля из датасета, неиспользованные автором этого блокнота.
#### Оценка качества рекомендательной системы
Цель рекомендательной системы - посоветовать пользователю объекты, которые он захочет приобрести. Для оценки качества такой системы мы воспользуемся метрикой `hit-ratio (HR)`. 

$$
HR = \frac{1}{|U_T|}\sum_{u \in U_T} \mathrm{I}(Rel_u \cap Rec_u)
$$

* $U_T$ - множество пользователей из тестовой выборки
* $Rec_u$ - множество объектов, рекомендованных пользователю $u$ 
* $Rel_u$ - множество объектов, оцененных пользователем $u$ в тестовой выборке
* $\mathrm{I}(Rel_u \cap Rec_u)$ - бинарная функция-индикатор. Функция возвращает 1 если $Rel_u \cap Rec_u \ne \emptyset$, иначе 0

$HR=1$ если для каждого пользователя мы рекомендовали хотя бы один релевантный объект. Так как обычно пользователи просматривают только первые $N$ рекомендаций, мы будем считать метрику $HR@N$, где $N=10$ (т.е. множество $Rec_u$ будет содержать только 10 объектов). 

# Условные обозначения
* `uid` - идентификатор пользователя
* `iid` - идентификатор объекта

# Games RSs

In [1]:
# импорты, которые точно понадобятся
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize, binarize

In [2]:
JSON_DATA_PATH = "data/Video_Games_5.json"
N = 10

## Анализ данных

In [3]:
import json

def iter_json_data(path):
    with open(path) as f:
        for line in f:
            data = json.loads(line)
            yield data
            
def get_data_frame():
    uid_to_id = {}
    iid_to_id = {}
    
    cols = ["uid", "iid", "review", "rating", "dt"]
    rows = []
    for d in iter_json_data(JSON_DATA_PATH):
        uid = uid_to_id.setdefault(d["reviewerID"], len(uid_to_id))
        iid = iid_to_id.setdefault(d["asin"], len(iid_to_id))
        review = d["reviewText"]
        rating = float(d["overall"])
        dt = int(d["unixReviewTime"])
        rows.append((uid, iid, review, rating, dt))
        
    return pd.DataFrame(rows, columns=cols)

In [4]:
df = get_data_frame()
df.head()

,uid,iid,review,rating,dt
0,0,0,Installing the game was a struggle (because of...,1.0,1341792000
1,1,0,If you like rally cars get this game you will ...,4.0,1372550400
2,2,0,1st shipment received a book instead of the ga...,1.0,1403913600
3,3,0,"I got this version instead of the PS3 version,...",3.0,1315958400
4,4,0,I had Dirt 2 on Xbox 360 and it was an okay ga...,4.0,1308009600


In [5]:
print("min-max количество объектов на пользователя:", 
      df.groupby("uid").iid.nunique().min(), df.groupby("uid").iid.nunique().max())
print("min-max количество пользователей на объект:", 
      df.groupby("iid").uid.nunique().min(), df.groupby("iid").uid.nunique().max())

min-max количество объектов на пользователя: 5 773
min-max количество пользователей на объект: 5 802


In [6]:
# проверяем, есть ли случаи, когда один и тот же пользователь оставляет отзывы на один и тот же объект
df.groupby(["uid", "iid"]).review.count().unique()  # ура, таких случаев нет

array([1], dtype=int64)

In [7]:
print("Количество объектов:", df.iid.unique().size)
print("Количество пользователей:", df.uid.unique().size)

Количество объектов: 10672
Количество пользователей: 24303


## Готовим выборки

In [8]:
def split_df_by_dt(df, p=0.8):
    """Функция разбивает df на тестовую и тренировочную выборки по времени 
    публикации отзывов (значение времени в поле dt)
    
    :param p: персентиль значений dt, которые образуют тренировочную выборку. Например p=0.8 означает, что в 
    тренировочной части будут отзывы, соответствующие первым 80% временного интервала 
    :return: два pd.DataFrame объекта
    """
    border_dt = df.dt.quantile(p)
    print("Min=%s, border=%s, max=%s" % (df.dt.min(), border_dt, df.dt.max()))
    training_df, test_df  = df[df.dt <= border_dt], df[df.dt > border_dt]
    print("Размер до очистки:", training_df.shape, test_df.shape)
    # удаляем из тестовых данных строки, соответствующие пользователям или объектам, 
    # которых нет в тренировочных данных 
    # (пользователи - избегаем проблем для персональных систем, объекты - для всех)
    test_df = test_df[test_df.uid.isin(training_df.uid) & test_df.iid.isin(training_df.iid)]
    print("Размер после очистки:", training_df.shape, test_df.shape)
    return training_df, test_df

In [9]:
training_df, test_df = split_df_by_dt(df)
del df

Min=939859200, border=1377129600.0, max=1405987200
Размер до очистки: (185427, 5) (46353, 5)
Размер после очистки: (185427, 5) (19174, 5)


In [10]:
def clean_df(df, min_review_per_uid, min_review_per_iid):
    """Функция удаляет из df строки, соответствующие пользователям и объектам, 
    у которых меньше min_review_per_uid и min_review_per_iid отзывов соответственно
    """
    _df = df.copy()
    while True:
        review_per_uid = _df.groupby("uid").review.count()
        bad_uids = review_per_uid[review_per_uid < min_review_per_uid].index
    
        review_per_iid = _df.groupby("iid").review.count()
        bad_iids = review_per_iid[review_per_iid < min_review_per_iid].index
        
        if bad_uids.shape[0] > 0 or bad_iids.shape[0] > 0:
            _df = _df[(~_df.uid.isin(bad_uids)) & (~_df.iid.isin(bad_iids))]
        else:
            break
    return _df

 ## Метрика

Для упрощения тестирования предлагается использовать словарь следующего типа:

```python
recs = {
    uid_1: {
        iid_1: score_11,
        iid_2: score_12,
        ...
    },
    uid_2: {
        iid_1: score_21,
        iid_2: score_22,
        ...
    },
    ...
}
```

где `uid_i` - идентификатор тестового пользователя, `iid_j` - идентификатор рекомендованного объекта, а `score_ij` - предсказанный рейтинг/вес объекта `j` для пользователя `i`.

In [11]:
def hit_ratio(recs_dict, test_dict):
    """Функция считает метрику hit-ration для двух словарей
    :recs_dict: словарь рекомендаций типа {uid: {iid: score, ...}, ...}
    :test_dict: тестовый словарь типа {uid: {iid: score, ...}, ...}
    """
    hits = 0
    for uid in test_dict:
        if set(test_dict[uid].keys()).intersection(recs_dict.get(uid, {})):
            hits += 1
    return hits / len(test_dict)

In [12]:
def get_test_dict(test_df):
    """Функция, конвертирующая тестовый df в словарь
    """
    test_dict = {}
    for t in test_df.itertuples():
        test_dict.setdefault(t.uid, {})
        test_dict[t.uid][t.iid] = t.rating
    return test_dict

test_dict = get_test_dict(test_df)

## Item-based collaborative filtering RS

Item-based CF основан на идее, что пользователь предпочтет объекты, похожие на те, что он приобретал ранее. Данные в CF модели представлены матрицей `user x item`, где ячейка матрицы соответствует рейтингу, который пользователь поставил объекту. Вместо рейтингов в матрице могут быть вероятности (т.е. вероятность, что пользователь воспользуется объектом). Для работы модели необходимо построить матрицу `item x item` схожести объектов. Обычно для построения матрицы схожести используется исходная матрица `user x item`. Чтобы уменьшить шумы в матрице схожести, для каждого объекта хранят только $K$ наиболее похожих объектов.

В простейшем случае рекомендации строятся путем нахождения объектов с наибольшим значением предсказанного рейтинга:
$$\hat{r}_{ui} = \frac{\sum_{j \in I_u} r_{uj} * sim(j, i)}{\sum_{j \in I_u} r_{uj}}$$

* $I_u$ - множество объектов, оцененных пользователем
* $sim(j, i)$ - схожесть между объектами $j$ и $i$

Часто из финальных рекомендаций для пользователя $u$ исключаются объекты $I_u$.

In [13]:
# вспомогательные функции, которые могут пригодиться при построении Item-based CF
def nullify_main_diagonal(m):
    positions = range(m.shape[0])
    eye = csr_matrix((np.ones(len(positions)), (positions, positions)), m.shape)
    return m - m.multiply(eye)


def get_topk(matrix, top, axis=1):
    """Converts source matrix to Top-K matrix
    where each row or column contains only top K values

    :param matrix: source matrix
    :param top: number of top items to be stored
    :param axis: 0 - top by column, 1 - top by row
    :return:
    """
    rows = []
    cols = []
    data = []

    if axis == 0:
        matrix = matrix.T.tocsr()

    for row_id, row in enumerate(matrix):
        if top is not None and row.nnz > top:
            top_args = np.argsort(row.data)[-top:]

            rows += [row_id] * top
            cols += row.indices[top_args].tolist()
            data += row.data[top_args].tolist()
        elif row.nnz > 0:
            rows += [row_id] * row.nnz
            cols += row.indices.tolist()
            data += row.data.tolist()

    topk_m = csr_matrix((data, (rows, cols)), (matrix.shape[0], matrix.shape[1]))

    if axis == 0:
        topk_m = topk_m.T.tocsr()

    return topk_m

#### `HR@10` для item-based CF модели, созданной автором блокнота: 0.085

### Подcказки
* Определитесь с тем, что вы пытаетесь предсказать (рейтинги, вероятности, ...)
* Оптимальный способ вычисления матрицы схожести выглядит так:
 * Привести строки в матрице `user x item` к единичной длине (выделяет основные предпочтения пользователя)
 * Построить матрицу схожести `item x item`
 * Для каждого объекта оставить только $K$ наиболее похожих объектов
 * Для каждого объекта привести к единичной длине вектор схожести этого объекта (выделяет наиболее схожие объекты)
* Удалили ли вы из рекомендаций объекты, которые пользователь уже оценивал?
* Статья "Item-Based Top-N Recommendation Algorithms", Mukund Deshpande и George Karypis

### Item-based collaborative filtering RS

Выбор модели обусловлен тем что Количество объектов: 10672, Количество пользователей: 24303 соответвенно нам лучше подойдет модель основанная на Item 

#### Построение матрицы схожести

In [14]:
#Удалим обькты которые имеют только одну оценку пользователя
training_df=clean_df(training_df,0,1)

In [15]:
training_df['rating'].std()

1.2051063261170754

In [16]:
training_df=training_df[['uid', 'iid','rating']]

In [17]:
training_df.query('uid==13')

,uid,iid,rating
33761,13,1903,5.0
43610,13,2455,5.0
43823,13,2470,3.0
70987,13,3872,5.0
111232,13,5782,5.0
146928,13,7156,5.0


In [18]:
# для построениея user_item

def load_data(df):
    rows = []
    cols = []
    data = []
    
    uid_to_row = {}
    iid_to_col = {}
    
    for t in df.itertuples():
        row_id = uid_to_row.setdefault(t.uid, len(uid_to_row))
        col_id = iid_to_col.setdefault(t.iid, len(iid_to_col))
        rating = t.rating
        
        rows.append(row_id)
        cols.append(col_id)
        data.append(rating)
        
    ui_m = csr_matrix((data, (rows, cols)))
    return ui_m, uid_to_row, iid_to_col


In [19]:
ui_m, uid_to_row, iid_to_col = load_data(training_df)
ui_m.shape


(22215, 10098)

In [205]:
# нормализуем по пользователям
n_ui_m = normalize(ui_m)

In [206]:
ui_m[0].data

array([ 1.,  5.,  1.,  5.,  1.,  2.])

In [207]:
n_ui_m[0].data

array([ 0.13245324,  0.66226618,  0.13245324,  0.66226618,  0.13245324,
        0.26490647])

### Z score

In [ ]:
item_avg_rating = ui_m.sum(axis=0) / binarize(ui_m).sum(axis=0)
item_avg_rating

In [ ]:
def get_sigmas(ui_m, item_avg_rating):
    n_ui_m = ui_m - binarize(ui_m).multiply(item_avg_rating)  # r_{ui} - \bar{r}_i
    n_ui_m.data = n_ui_m.data ** 2  # (r_{ui} - \bar{r}_i)^2
    n_ui_m_sum = n_ui_m.sum(axis=0)  # \sum(r_{ui} - \bar{r}_i)^2
    ratings_per_item = binarize(ui_m).sum(axis=0)  # |U_{i}|
    sigmas = np.sqrt(n_ui_m_sum / ratings_per_item)
    return sigmas

In [ ]:
sigmas = get_sigmas(ui_m, item_avg_rating)
sigmas

In [ ]:
inv_sigmas = 1 / sigmas
z_score= (ui_m - binarize(ui_m).multiply(item_avg_rating)).multiply(inv_sigmas)
z_score=z_score.tocsr()

In [ ]:
n_ui_m=z_score
n_ui_m = normalize(ui_m)

### Mean-centering

In [20]:
# мы хотим увидеть средний среди существующих рейтингов
user_avg_rating = ui_m.sum(axis=1) / binarize(ui_m).sum(axis=0)
user_avg_rating

matrix([[  0.75      ,   3.        ,   2.5       , ...,  15.        ,
           7.5       ,   5.        ],
        [  0.85      ,   3.4       ,   2.83333333, ...,  17.        ,
           8.5       ,   5.66666667],
        [  1.6       ,   6.4       ,   5.33333333, ...,  32.        ,
          16.        ,  10.66666667],
        ..., 
        [  0.25      ,   1.        ,   0.83333333, ...,   5.        ,
           2.5       ,   1.66666667],
        [  0.25      ,   1.        ,   0.83333333, ...,   5.        ,
           2.5       ,   1.66666667],
        [  0.25      ,   1.        ,   0.83333333, ...,   5.        ,
           2.5       ,   1.66666667]])

In [21]:
m_c = ui_m - binarize(ui_m).multiply(user_avg_rating)
n_ui_m = normalize(m_c)

#### матрица схожести Item-Item 

In [22]:
# находим матрицу схожести item-item
from sklearn.metrics.pairwise import cosine_similarity
ii_sim_m = cosine_similarity(n_ui_m.T, dense_output=False).tocsr()

In [23]:
TOP=30
# храним только top-30
ii_sim_m = nullify_main_diagonal(ii_sim_m)
ii_sim_m = get_topk(ii_sim_m, TOP)

In [24]:
# нормализуем схожести
ii_sim_m = normalize(ii_sim_m)

## построим предсказание

In [25]:
test_df.query('uid==13').head(10)

,uid,iid,review,rating,dt
13,13,0,I bought this and the key didn't work. It was...,1.0,1404086400
131223,13,6607,I used these replacement cables to repair a he...,5.0,1404086400
215998,13,10044,If there are any PC games that can rival both ...,5.0,1389052800
220011,13,10129,This expansion / DLC is a must-have for Civ 5....,5.0,1404086400


In [26]:
col_to_iid = {col_id: iid for iid, col_id in iid_to_col.items()}

In [31]:
def get_recs(uid, top):
    recs = {}
    if uid in uid_to_row:
        u_row_id = uid_to_row[uid]
        
        recsk = n_ui_m[u_row_id].dot(ii_sim_m.T)
        
        b_bnr=(binarize(n_ui_m[u_row_id])*100).tocsr()
              
        #recsk=recsk-b_bnr
        
        #iid_to_remove=training_df[training_df.uid==uid]['iid'].unique()
        # и узнаем их количество
        #cnt_tu_del=len(iid_to_remove)            

        #sort_indices = np.argsort(recsk.data)[::-1]
        
        for arg_id in np.argsort(recsk.data)[-top:][::-1]:
            iid = recsk.indices[arg_id]
            riid=col_to_iid[iid]
            score = recsk.data[arg_id]
            #if riid not in iid_to_remove: 
            recs[riid] = score
            #if len(recs)>=top:
            #    return recs
                  
    return recs

In [32]:
get_recs(13, 7)

{8: 0.1529839419549861,
 2338: 0.15225232413820744,
 2339: 0.16965551556850625,
 3598: 0.17756010950506632,
 7917: 0.16911384687637765,
 8314: 0.19075387755697742,
 8821: 0.18504525495879684}

In [33]:
    def get_batch_recs( uids, top):
        return {uid: get_recs(uid, top) for uid in uids}

In [34]:
%%time
rec_dict=get_batch_recs(test_df.uid,7)

Wall time: 2min 34s


In [35]:
# тупо берем первые 10 по суммарным баллам за просмотр
hit_ratio(rec_dict,test_dict)

0.04900953778429934

In [41]:
%%time
# Сделаем перебор по количеству попаданий из CF
n_hr={}
for i in range(4,10,1):
    rec_dict=get_batch_recs(test_df.uid,i)
    n_hr[i]=hit_ratio(rec_dict,test_dict)  

Wall time: 15min 52s


In [ ]:
# результат НК от максимума
n_hr 

## Выводы

#### Вариант по умолчанию:  Храним 30,  HR:0.062

#### Вариант 1:  Храним 30,  Z-score  HR: 0.062

#### Вариант 2:  Храним 30,  Mean-centering  HR: 0.090

#### Вариант 3:  Храним 30,  Mean-centering + Удаление уже просмотренных  HR: 0.72 . странный результат, объяснить не могу, переделал по другому все равно   HR:0.073. хуже

#### Вариант 4:  Храним 50,  Mean-centering   HR: 0.089


#### Вариант 5:  Храним 20,  Mean-centering   HR: 0.089